# Agrupación y Segmentación de Vecindarios en Toronto
## 2. Explorar Barrios en Toronto

Explore y agrupe los vecindarios de Toronto. Puede decidir trabajar sólo con los municipios que contienen la palabra Toronto y luego replicar el mismo análisis que hicimos con los datos de la ciudad de Nueva York. Es su decisión.

Sólo asegúrese:

1.     Agregar suficientes celdas de Anotación para explicar lo que decidió hacer y para informar cualquier observación que haga.


Primero, importamos las librerías necesarias.
Después leemos los resultados del ejercicio anterior.

In [1]:

# Importamos "Numpy" como 'np'
import numpy as np
# Importamos "Pandas" como 'pd'
import pandas as pd
# Importamos 'fsqApiKey' desde "config"
from config import fsqApiKey
# Importamos "Requests"
import requests
# Leemos el archivo del ejercicio anterior y lo asignamos a la variable "df_merged"
df_merged = pd.read_csv("Toronto_Q2.csv")

### Funciones predefinidas en Labs anteriores

- He añadido líneas de código de control de flujo para depurar errores previos.

In [2]:
# Función "getNearbyVenues"
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 50
    
    headers = {
        'Accept': 'application/json',
        'Authorization': fsqApiKey
    }

    URL = 'https://api.foursquare.com/v3/places/search?ll={},{}&radius={}&limit={}'

    venues_list = []
    

    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = URL.format(lat, lng, radius, LIMIT)
        response = requests.get(url, headers=headers)       
    
        if response.status_code != 200:
            # La respuesta del servidor no es correcta, mostramos un mensaje y pasamos al siguiente vecindario
            print(f"Error al obtener datos para el vecindario {name}: {response.status_code}")
            continue
        data = response.json()

        if 'results' not in data:
        # La respuesta no contiene una clave 'results', mostramos un mensaje y pasamos al siguiente vecindario
            print(f"Error al obtener datos para el vecindario {name}: la respuesta no contiene resultados")
            continue
        # Procesamos venues_list
        for record in data['results']:
            if not record['categories']:
                # La lista está vacía, ignoramos el registro actual
                print(record['name'] + " no tiene categoría")
                continue
            else:
                venues_list.append({
                    'Vecindario': name,
                    'Latitud': lat,
                    'Longitud': lng,
                    'Venue': record['name'],
                    'Venue Latitude': record['geocodes']['main']['latitude'],
                    'Venue Longitude': record['geocodes']['main']['longitude'],
                    'Venue Category': record['categories'][0]['name'],
                })

    # Creamos el dataFrame 'nearby_venues' con 'venues_list'
    column_names = ['Vecindario', 
                  'Latitud', 
                  'Longitud', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    nearby_venues = pd.DataFrame(data=venues_list , columns=column_names)

    return(nearby_venues)


# Función que nos devuelve la categoría de cada sitio
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# Función quenos devuelve los lugares con más puntuación
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Importamos el resto de librerías que nos faltan:

In [3]:
# Importamos la librería JSON
import json 

# Instalamos geopy (En este caso ya está instalado, en caso contrario deshaz el comentario)
# c!onda install -c conda-forge geopy --yes 

# Importamos "Nominatim" desde "geopy.geocoders" para convertir una direción en coordenadas
from geopy.geocoders import Nominatim

# Importamos "Matplotlib.cm" como 'cm'
import matplotlib.cm as cm
# Importamos "Matplotlib.colors" como 'colors'
import matplotlib.colors as colors

# Importamos "KMeans" desde "sklearn.cluster"
from sklearn.cluster import KMeans

# Importamos "Folium" (En este caso ya está instalado, en caso contrario deshaz el comentario)
# c!onda install -c conda-forge folium=0.5.0 --yes

#Importamos "Folium"
import folium

print('✅  Todas las librerías se han importado correctamente ✅')

✅  Todas las librerías se han importado correctamente ✅


2.     Generar mapas para visualizar sus vecindarios y cómo se agrupan.

Dibujamos un mapa de Toronto con todos los Vecindarios:

In [4]:

#43.628841 	-79.520999
latitude = 43.628841
longitude = -79.520999


# create map of New York using latitude and longitude values
mapa_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, municipio, vecindario in zip(df_merged['Latitud'], df_merged['Longitud'], df_merged['Municipio'], df_merged['Vecindario']):
    label = '{}, {}'.format(vecindario, municipio)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapa_toronto)  
    
mapa_toronto



Cogemos sólo los datos de las filas que contengan "Toronto"

In [5]:
datos = df_merged.set_index('Municipio').filter(regex=".*Toronto",axis=0)
datos.head()



,CódigoPostal,Vecindario,Latitud,Longitud
Municipio,,,,
Downtown Toronto,M5A,Regent Park / Harbourfront,43.654260,-79.360636
Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937
Downtown Toronto,M5C,St. James Town,43.651494,-79.375418
East Toronto,M4E,The Beaches,43.676357,-79.293031
Downtown Toronto,M5E,Berczy Park,43.644771,-79.373306


Usamos "getNearbyVenues" con los datos de Toronto.

In [6]:



venuesToronto = getNearbyVenues(names = datos['Vecindario'],
                                   latitudes = datos['Latitud'],
                                   longitudes = datos['Longitud'])

Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Tscc 1930 no tiene categoría
Sold in a Day no tiene categoría
Christopher Lawson Photography no tiene categoría
The Nail Market no tiene categoría
Celestial Stain Glass Li Inc no tiene categoría
Eaune Natural Skin Care Studio no tiene categoría
Berczy Park
Central Bay Street
Christie
Sporometrics Inc no tiene categoría
Marquees Tents no tiene categoría
Unofficial: Etta no tiene categoría
Gelico Gallery no tiene categoría
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Rock & Rose no tiene categoría
SoundFX no tiene categoría
The Danforth East
Drift Massage Therapy no tiene categoría
Localcoin Bitcoin ATM - Sammon Convenience no tiene categoría
East York Handyman no tiene categoría
Dupere Casting no tiene categoría
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
TODO Fusion Resto-Bar no tiene categoría
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchan

In [7]:
print('El tamaño del dataFrame es de {}'.format(venuesToronto.shape))
venuesToronto.head()
venuesToronto.groupby('Vecindario').count()
print('Hay {} categorías únicas'.format(len(venuesToronto['Venue Category'].unique())))

El tamaño del dataFrame es de (1712, 7)
Hay 314 categorías únicas


## 3. Analizar Cada Barrio


Hacemos One Hot Encoding

In [8]:
# Codificación One Hot
toronto_onehot = pd.get_dummies(venuesToronto[['Venue Category']], prefix="", prefix_sep="")

# Añadimos la columna 'Vecindario' de vuelta al dataframe
toronto_onehot['Vecindario'] = venuesToronto['Vecindario'] 

# Movemos la columna 'Vecindario' a la primera columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Vecindario,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Store,Architecture Firm,Argentinian Restaurant,...,Used Car Dealership,Vegan and Vegetarian Restaurant,Video Games Store,Video Store,Vietnamese Restaurant,Vintage and Thrift Store,Website Designer,Wine Bar,Women's Store,Yoga Studio
0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Agrupamos las filas por barrios tomando la media de la frecuencia de cada categoría

In [9]:
toronto_agrupado = toronto_onehot.groupby('Vecindario').mean().reset_index()
toronto_agrupado

,Vecindario,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Store,Architecture Firm,Argentinian Restaurant,...,Used Car Dealership,Vegan and Vegetarian Restaurant,Video Games Store,Video Store,Vietnamese Restaurant,Vintage and Thrift Store,Website Designer,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.02,0.020000,0.000000,0.000000,0.000000,...,0.00,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.000000,0.063830,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Christie,0.000000,0.000000,0.021739,0.000000,0.00,0.021739,0.021739,0.021739,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000
5,Church and Wellesley,0.020000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Commerce Court / Victoria Hotel,0.000000,0.000000,0.020000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.00,0.020408,0.020408,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.022222,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000
9,Dufferin / Dovercourt Village,0.000000,0.020833,0.020833,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Examinamos el tamaño del nuevo dataframe

In [10]:
toronto_agrupado.shape

(39, 315)

#### Imprimimos cada barrio junto con los 5 sitios más comunes

In [11]:
num_top_venues = 5

for hood in toronto_agrupado['Vecindario']:
    print("----"+hood+"----")
    temp = toronto_agrupado[toronto_agrupado['Vecindario'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Berczy Park----
              venue  freq
0               Pub  0.06
1          Beer Bar  0.04
2   Farmers' Market  0.04
3         Drugstore  0.04
4  Sushi Restaurant  0.04


----Brockton / Parkdale Village / Exhibition Place----
                venue  freq
0  Advertising Agency  0.06
1          Hair Salon  0.06
2                Café  0.04
3  Photography Studio  0.04
4       Design Studio  0.04


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
                         venue  freq
0       Landmarks and Outdoors  0.14
1              Harbor / Marina  0.14
2     Home Improvement Service  0.14
3  Historic and Protected Site  0.14
4        Sports and Recreation  0.14


----Central Bay Street----
                           venue  freq
0  Cafes, Coffee, and Tea Houses  0.12
1                    Coffee Shop  0.08
2                           Café  0.06
3                           Deli  0.06
4                          Din

Generamos el nuevo dataframe y mostramos los primeros 10 sitios de cada barrio.
Hacemos esto para prepararlo para el cluster analysis.

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Vecindario']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Lugar más común'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}º Lugar más común'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Vecindario'] = toronto_agrupado['Vecindario']

for ind in np.arange(toronto_agrupado.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_agrupado.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Vecindario,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4º Lugar más común,5º Lugar más común,6º Lugar más común,7º Lugar más común,8º Lugar más común,9º Lugar más común,10º Lugar más común
0,Berczy Park,Pub,Beer Bar,Farmers' Market,Drugstore,Sushi Restaurant,Night Club,Fish and Chips Shop,Coffee Shop,Park,Greek Restaurant
1,Brockton / Parkdale Village / Exhibition Place,Advertising Agency,Hair Salon,Café,Photography Studio,Design Studio,Laundry Service,Electronics Store,Italian Restaurant,Retirement Home,Financial Service
2,CN Tower / King and Spadina / Railway Lands / ...,Landmarks and Outdoors,Harbor / Marina,Home Improvement Service,Historic and Protected Site,Sports and Recreation,Sculpture Garden,Property Management Office,Music Venue,Music Store,Music School
3,Central Bay Street,"Cafes, Coffee, and Tea Houses",Coffee Shop,Café,Deli,Diner,Pizzeria,Japanese Restaurant,Juice Bar,Thai Restaurant,Fast Food Restaurant
4,Christie,Grocery Store / Supermarket,Automotive Repair Shop,Hardware Store,Café,Hair Salon,Restaurant,Massage Clinic,Gym and Studio,Metals Supplier,Gift Store


## 4. Barrios Agrupados

Ejecutamos k-means para agrupar los vecindarios en 5 agrupaciones.
Después generamos un nuevo dataframe que incluya la agrupación así como los 10 sitios más populares de cada barrio.

In [21]:
# establecer el número de agrupaciones
kclusters = 5

toronto_grouped_clustering = toronto_agrupado.drop('Vecindario', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10]


# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = datos

# juntar toronto_grouped con datos 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Vecindario'), on='Vecindario')

toronto_merged.head() # revisar las ultimas columnas

/var/folders/g3/6cjht3vd44sb4hzwvmrc_lt00000gn/T/ipykernel_78214/988763815.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  toronto_grouped_clustering = toronto_agrupado.drop('Vecindario', 1)


,CódigoPostal,Vecindario,Latitud,Longitud,Cluster Labels,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4º Lugar más común,5º Lugar más común,6º Lugar más común,7º Lugar más común,8º Lugar más común,9º Lugar más común,10º Lugar más común
Municipio,,,,,,,,,,,,,,,
Downtown Toronto,M5A,Regent Park / Harbourfront,43.654260,-79.360636,0,Car Dealership,Bakery,Restaurant,Park,Coffee Shop,Italian Restaurant,Automotive Repair Shop,Design Studio,Furniture and Home Store,Music Venue
Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937,3,Fast Food Restaurant,Department Store,Clothing Store,Bookstore,"Cafes, Coffee, and Tea Houses",Café,Movie Theater,Coffee Shop,Beer Bar,Exhibit
Downtown Toronto,M5C,St. James Town,43.651494,-79.375418,3,Café,Coffee Shop,Pizzeria,Diner,BBQ Joint,Asian Restaurant,Creperie,Night Club,Sports Bar,Speakeasy
East Toronto,M4E,The Beaches,43.676357,-79.293031,0,Interior Designer,Hair Salon,Dance Studio,Park,"Shipping, Freight, and Material Transportation...",Grocery Store / Supermarket,Boutique,Massage Clinic,General Contractor,Burger Joint
Downtown Toronto,M5E,Berczy Park,43.644771,-79.373306,3,Pub,Beer Bar,Farmers' Market,Drugstore,Sushi Restaurant,Night Club,Fish and Chips Shop,Coffee Shop,Park,Greek Restaurant


Por último, visualizamos los vecindarios que son similares en función de los tipos de lugares que tienen con más frecuencia.

In [14]:
# Creamos el mapa
latitude = 43.654260
longitude = -79.520999

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# Seteamos los colores para los clústeres
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Añadimos marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitud'], toronto_merged['Longitud'], toronto_merged['Vecindario'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examinar Agrupaciones

Vamos a examinar cada agrupación y determinar las categorías del sitio que distingue a cada agrupación.

#### Agrupación 1

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Vecindario,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4º Lugar más común,5º Lugar más común,6º Lugar más común,7º Lugar más común,8º Lugar más común,9º Lugar más común,10º Lugar más común
Municipio,,,,,,,,,,,
Downtown Toronto,Regent Park / Harbourfront,Car Dealership,Bakery,Restaurant,Park,Coffee Shop,Italian Restaurant,Automotive Repair Shop,Design Studio,Furniture and Home Store,Music Venue
East Toronto,The Beaches,Interior Designer,Hair Salon,Dance Studio,Park,"Shipping, Freight, and Material Transportation...",Grocery Store / Supermarket,Boutique,Massage Clinic,General Contractor,Burger Joint
Downtown Toronto,Christie,Grocery Store / Supermarket,Automotive Repair Shop,Hardware Store,Café,Hair Salon,Restaurant,Massage Clinic,Gym and Studio,Metals Supplier,Gift Store
West Toronto,Dufferin / Dovercourt Village,Automotive Repair Shop,Liquor Store,Diner,Hair Salon,Bar,Food and Beverage Service,Grocery Store / Supermarket,Sports and Recreation,Speakeasy,Middle Eastern Restaurant
East YorkEast Toronto,The Danforth East,Convenience Store,Park,Landscaper and Gardener,Print Store,Bathroom Contractor,Media Agency,Photographer,Business and Professional Services,Clothing Store,Barbershop
West Toronto,Little Portugal / Trinity,Wine Bar,Bar,Music Venue,Art Gallery,Asian Restaurant,Diner,French Restaurant,Bakery,Retail,Coffee Shop
East Toronto,The Danforth West / Riverdale,Restaurant,Bakery,Ice Cream Parlor,Greek Restaurant,Italian Restaurant,Diner,Café,Bookstore,Hair Salon,Juice Bar
West Toronto,Brockton / Parkdale Village / Exhibition Place,Advertising Agency,Hair Salon,Café,Photography Studio,Design Studio,Laundry Service,Electronics Store,Italian Restaurant,Retirement Home,Financial Service
East Toronto,India Bazaar / The Beaches West,Pizzeria,Pet Grooming Service,Café,Bicycle Store,Convenience Store,Electronics Store,Business and Professional Services,Park,Flower Store,Grocery Store / Supermarket


#### Agrupación 2

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Vecindario,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4º Lugar más común,5º Lugar más común,6º Lugar más común,7º Lugar más común,8º Lugar más común,9º Lugar más común,10º Lugar más común
Municipio,,,,,,,,,,,
Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,Landmarks and Outdoors,Harbor / Marina,Home Improvement Service,Historic and Protected Site,Sports and Recreation,Sculpture Garden,Property Management Office,Music Venue,Music Store,Music School


#### Agrupación 3

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Vecindario,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4º Lugar más común,5º Lugar más común,6º Lugar más común,7º Lugar más común,8º Lugar más común,9º Lugar más común,10º Lugar más común
Municipio,,,,,,,,,,,
Central Toronto,Forest Hill North & West,General Contractor,Business and Professional Services,"Heating, Ventilating and Air Conditioning Cont...",Management Consultant,Event Service,Restaurant,Liquor Store,Bar,"Cafes, Coffee, and Tea Houses",Bank
Downtown Toronto,Rosedale,Park,Interior Designer,Carpet and Flooring Contractor,Playground,Health and Beauty Service,Business and Professional Services,Framing Store,Caterer,Sports and Recreation,Loans Agency


#### Agrupación 4

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Vecindario,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4º Lugar más común,5º Lugar más común,6º Lugar más común,7º Lugar más común,8º Lugar más común,9º Lugar más común,10º Lugar más común
Municipio,,,,,,,,,,,
Downtown Toronto,"Garden District, Ryerson",Fast Food Restaurant,Department Store,Clothing Store,Bookstore,"Cafes, Coffee, and Tea Houses",Café,Movie Theater,Coffee Shop,Beer Bar,Exhibit
Downtown Toronto,St. James Town,Café,Coffee Shop,Pizzeria,Diner,BBQ Joint,Asian Restaurant,Creperie,Night Club,Sports Bar,Speakeasy
Downtown Toronto,Berczy Park,Pub,Beer Bar,Farmers' Market,Drugstore,Sushi Restaurant,Night Club,Fish and Chips Shop,Coffee Shop,Park,Greek Restaurant
Downtown Toronto,Central Bay Street,"Cafes, Coffee, and Tea Houses",Coffee Shop,Café,Deli,Diner,Pizzeria,Japanese Restaurant,Juice Bar,Thai Restaurant,Fast Food Restaurant
Downtown Toronto,Richmond / Adelaide / King,Café,Restaurant,Pizzeria,Deli,Lounge,Sushi Restaurant,Music Venue,Burger Joint,"Cafes, Coffee, and Tea Houses",Salad Restaurant
Downtown Toronto,Harbourfront East / Union Station / Toronto Is...,Music Venue,Coffee Shop,Pizzeria,Park,Café,"Cafes, Coffee, and Tea Houses",Plaza,Sports Bar,Bar,Deli
Downtown Toronto,Toronto Dominion Centre / Design Exchange,Restaurant,Bank,Bakery,"Cafes, Coffee, and Tea Houses",Deli,Café,Burger Joint,Diner,Coffee Shop,Clothing Store
Downtown Toronto,Commerce Court / Victoria Hotel,Restaurant,Café,Bakery,"Cafes, Coffee, and Tea Houses",Coffee Shop,Pub,Bank,Burger Joint,Bistro,Diner
Downtown Toronto,Kensington Market / Chinatown / Grange Park,Café,Chinese Restaurant,Bakery,Vietnamese Restaurant,Barbershop,Butcher,Coffee Shop,Tattoo Parlor,Organic Grocery,Farmers' Market


#### Agrupación 5

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Vecindario,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4º Lugar más común,5º Lugar más común,6º Lugar más común,7º Lugar más común,8º Lugar más común,9º Lugar más común,10º Lugar más común
Municipio,,,,,,,,,,,
Central Toronto,Lawrence Park,Business and Strategy Consulting Office,Architecture Firm,Dim Sum Restaurant,Spa,Management Consultant,Legal Service,Bank,Tennis Club,Organization,Publisher
Central Toronto,Roselawn,Business and Professional Services,Business and Strategy Consulting Office,Computer Repair Service,Construction,"Shipping, Freight, and Material Transportation...",Nutritionist,Candy Store,Professional Cleaning Service,Mover,General Contractor
Central Toronto,Moore Park / Summerhill East,Business and Strategy Consulting Office,Financial Planner,Business and Professional Services,Architecture Firm,Park,Film Studio,"Shipping, Freight, and Material Transportation...",Retail,Real Estate Agency,Organization


## Conclusión

La población de Toronto se agrupa principalmente en 5 zonas:
- El clúster 0 son barrios predominantemente resideciales, de clase media y con todo tipo de servicios.
- El clúster 1 es donde se ubica el aeropuerto y el puerto, con zonas de esparcimiento. Ideal para turismo.
- El clúster 2 es el centro financiero, empresarial y de organismos oficiales. Se encuentra en el punto medio entre el centro y los suburbios.
- El cluster 3 es el núcleo de la ciudad donde se desarrolla el ocio y la restauración, y predominan los restaurantes.
- El cluster 4 es un barrio residencial exclusivo, de alto nivel económico con servicios propios.

Una vez que esté satisfecho con su análisis, envíe un enlace al nuevo Notebook en su repositorio de Github. (3 puntos)